In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
import pandas as pd
from selenium.webdriver.chrome.options import Options
import requests
import json
from config import geoapify_key
import re
import numpy as np


browser = Browser('chrome')
browser.driver.maximize_window()

In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card&f:@lcbo_current_offer=[On%20Sale]&f:@ec_category=[Wine]"
browser.visit(url)

# To select "English" as preffered language
time.sleep(3)
results = browser.links.find_by_text('English')
results.click()
time.sleep(3)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

In [3]:
# Removes the cookie popup
cookie = browser.find_by_id('btn-cookie-allow').click()

In [4]:
iteration = 0
while True:
    try:     
        # Wait for 1 second
        time.sleep(2)
        # Click on Load More button
        browser.find_by_id('loadMore').click()
    except:
        break
    print(iteration)
    iteration += 1

0
1
2
3


In [5]:
html = browser.html
    
soup = BeautifulSoup(html, 'html.parser')

all_links = soup.find_all("div", class_ = 'coveo-product-items')

links = []

for link in all_links:
    results = link.a['href']
    links.append(results)        

In [6]:
print(len(links))


212


In [7]:
# div_element = soup.find('div', class_='fotorama__stage__frame')
# if div_element and 'href' in div_element.attrs:
#     link = div_element['href']
#     print(link)
# else:
#     print("Link not found.")

In [8]:
regular_price = []
sale_price = []
name = []
rating = []
onsale = [] #this is the On Sale tag
reviews = []
size = []
description = []
category = []
details_list = []
taste_list = []
images = []


max_iteration2 = len(links)
iteration2 = 0


for record in links:
    if iteration2 >= max_iteration2:
        break
    url = record
    browser.visit(url)
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
#     time.sleep(1)
#     browser.find_by_css('my_store_close_button').click()

    try:
        div_element = soup.find('div', class_='fotorama__stage__frame')
        if div_element and 'href' in div_element.attrs:
            link = div_element['href']
            images.append(link)
        else:
            print("Link not found.")
    except AttributeError:
        images.append('N/A')
        
    try:
        old_price = soup.find('span', class_ = 'old-price')
        old = old_price.find('span', class_ = 'price')
        regular_price.append(old.find('span', class_ = 'price').text)
    except AttributeError:
        regular_price.append('N/A')

    try:
        on_sale_price = soup.find('span', class_ = 'special-price')
        new_price = on_sale_price.find('span', class_ = 'price')
        sale_price.append(new_price.find('span', class_ = 'price').text)
    except AttributeError:
        sale_price.append('N/A')
    
    try:    
        name.append(soup.find('div', class_ = 'page-title-wrapper').text)
    except AttributeError:
        name.append('N/A')
        
    try:
        onsale.append(soup.find('div', class_ = 'amlabel-text').text)
    except AttributeError:
        onsale.append('N/A')
    
    try:
        rating.append(soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text)
    except AttributeError:
        rating.append('N/A')
    
    try:
        reviews.append(soup.find('div', class_ = 'bv_numReviews_text').text)
    except AttributeError:
        reviews.append('N/A')
    
    try:
        size.append(soup.find('div', class_ = 'lcbo-product-size').text)
    except AttributeError:
        size.append('N/A')
        
    try:
        description.append(soup.find('div', class_ = 'testing_note').text)
    except AttributeError:
        description.append('N/A')
        
    categories_all = soup.find('div', class_ = 'breadcrumbs')
    categories = categories_all.find_all('li', class_ = 'item')
    category_list = []

    for category_record in categories:
        category_list.append(category_record.text)
    category.append(category_list[-2])

    moredetail = soup.find('div', class_ = 'moredetail')
    all_details = moredetail.find_all('li')
    details = {}

    for detail_record in all_details:
        details_value = detail_record.find('div', class_ = 'value').text
        details_name = detail_record.find('div', class_ = 'label').text
        details[details_name] = details_value

    details_list.append(details)
    
#     try:
#         flavour_all = soup.find('div', class_ = 'foodParings pip-info')
#         flavour = flavour_all.find('div', class_ = 'label').text
#         flavour_value = flavour_all.find('div', class_ = 'value').text
#         flavours = {
#             flavour: flavour_value
#         }
#     except AttributeError:
#         flavours = {
#             'Flavour': 'N/A'
#         }
        
    
#     try:
#         all_taste = soup.find('div', class_ = 'foodParings pip-info')
#         li_all = all_taste.find_all('li')

#         for li in li_all:
#             label = li.find('div', class_='label').text.strip()
#             value_element = li.find('div', class_='value')
#             if 'aria-label' in value_element.attrs:
#                 aria_label = value_element['aria-label']
#                 number = re.findall(r'\d+', aria_label)
#                 if number:
#                     taste_list.append({
#                         label: int(number[0])
#                     })
#     except AttributeError:
#         taste_list.append('N/A')
        
    print(iteration2)
    iteration2 += 1



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [67]:
data = {
    'Name': name,
    'Regular Price': regular_price,
    'Sale Price': sale_price,
    'Rating': rating,
    'Reviews': reviews,
    'Size': size,
    'Description': description,
    'Category': category,
    'Details': details_list,
    'Image': images,
    'Link': links
#     'Flavours': flavours
    
}

# Add the details dictionary to the data dictionary
data['Details'] = details_list
# data['Taste'] = taste_list

# Create a dataframe from the data dictionary
df = pd.DataFrame(data)

details_df = pd.json_normalize(df['Details'])
# taste_df = pd.json_normalize(df['Taste'])

df = pd.concat([df, details_df], axis = 1)
# df2 = pd.concat([df1, taste_df], axis = 1)

df.drop('Details', axis = 1, inplace = True)
# df2.drop('Taste', axis = 1, inplace = True)

#Adding Latitude and Longitude columns to the Data Frame
df['Lat'] = ""
df['Lon'] = ""   

In [68]:
print(len(regular_price))
print(len(sale_price))
print(len(name))
print(len(rating))
print(len(onsale))
print(len(reviews))
print(len(size))
print(len(description))
print(len(category))
print(len(details_list))
print(len(images))
print(len(links))
# print(len(flavours))
# print(flavours)

212
212
212
212
212
212
212
212
212
212
212
212


In [69]:
#API call to get Latitude and Longitude
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

df['Lat'] = None
df['Lon'] = None

for index, country in df['Made In'].to_frame().iterrows():
    location = country['Made In']
    params["text"] = location
    
    response = requests.get(base_url, params=params).json()

    latitude = response["results"][0]["lat"]
    longitude = response["results"][0]["lon"]
    df.loc[index, "Lat"] = latitude
    df.loc[index, "Lon"] = longitude

df.head()


,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Image,Link,Alcohol/Vol,Made In,Sugar Content,Varietal,By,Release Date,Lat,Lon
0,Paul Mas Cabernet Sauvignon,$14.95,$12.95,N/A,(0),750 ml bottle,Paul Mas vineyards are in the Languedoc region...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/paul-mas-cabernet-sauv...,13.5%,"Languedoc, France",5 g/L,Cabernet Sauvignon,NaN,NaN,47.218868,5.965036
1,Blu Giovello Prosecco,$18.00,$15.50,4.4,(9),750 ml bottle,The prosecco in the distinctive blue bottle. I...,White Sparkling Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/blu-giovello-prosecco-...,11%,Italy,14 g/L,Prosecco,Blu Giovello,NaN,42.638426,12.674297
2,Bask Cabernet Sauvignon,$13.45,$12.45,3.0,(6),750 ml bottle,Low in sugar and high in flavour! Bask created...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/bask-cabernet-sauvigno...,12.5%,Canada,3 g/L,Cabernet Sauvignon,NaN,NaN,61.066692,-107.991707
3,Jacob's Creek Moscato Rosé,$16.00,$14.00,4.7,(18),750 ml bottle,"Jacob's Creek Moscato is a fresh and fragrant,...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/jacob-s-creek-moscato-...,7.5%,Australia,70 g/L,Rose Wines,Jacob's Creek,NaN,-24.776109,134.755
4,Sandbanks Summer Rosé VQA,$14.95,$12.95,N/A,(0),750 ml bottle,"Established in Prince Edward County in 2001, S...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/sandbanks-summer-ros-c...,11.6%,"Ontario, Canada",27 g/L,Rosé,Sandbanks,NaN,50.000678,-86.000977


In [78]:
df_copy = df
df_copy.head()

,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Image,Link,Alcohol/Vol,Made In,Sugar Content,Varietal,By,Release Date,Lat,Lon,Savings
0,Paul Mas Cabernet Sauvignon,14.95,12.95,N/A,(0),750 ml bottle,Paul Mas vineyards are in the Languedoc region...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/paul-mas-cabernet-sauv...,0.135,"Languedoc, France",5 g/L,Cabernet Sauvignon,NaN,NaN,47.218868,5.965036,2.0
1,Blu Giovello Prosecco,18.00,15.50,4.4,(9),750 ml bottle,The prosecco in the distinctive blue bottle. I...,White Sparkling Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/blu-giovello-prosecco-...,0.110,Italy,14 g/L,Prosecco,Blu Giovello,NaN,42.638426,12.674297,2.5
2,Bask Cabernet Sauvignon,13.45,12.45,3.0,(6),750 ml bottle,Low in sugar and high in flavour! Bask created...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/bask-cabernet-sauvigno...,0.125,Canada,3 g/L,Cabernet Sauvignon,NaN,NaN,61.066692,-107.991707,1.0
3,Jacob's Creek Moscato Rosé,16.00,14.00,4.7,(18),750 ml bottle,"Jacob's Creek Moscato is a fresh and fragrant,...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/jacob-s-creek-moscato-...,0.075,Australia,70 g/L,Rose Wines,Jacob's Creek,NaN,-24.776109,134.755,2.0
4,Sandbanks Summer Rosé VQA,14.95,12.95,N/A,(0),750 ml bottle,"Established in Prince Edward County in 2001, S...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/sandbanks-summer-ros-c...,0.116,"Ontario, Canada",27 g/L,Rosé,Sandbanks,NaN,50.000678,-86.000977,2.0


In [76]:
df['Regular Price'] = df['Regular Price'].str.replace('N/A','0')
df['Sale Price'] = df['Sale Price'].str.replace('N/A','0')
df['Regular Price'] = df['Regular Price'].str.replace('$','')
df['Sale Price'] = df['Sale Price'].str.replace('$','')





df['Savings'] = df['Regular Price'].astype(float) - df['Sale Price'].astype(float)

df['Alcohol/Vol'] = df['Alcohol/Vol'].str.rstrip('%')
df['Alcohol/Vol'] = pd.to_numeric(df['Alcohol/Vol']) /100
df['Image'] = df['Image'].str.replace('1280.1280','319.319')

df.head()

C:\Users\josed\AppData\Local\Temp\ipykernel_16196\3373048541.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Regular Price'] = df['Regular Price'].str.replace('$','')
C:\Users\josed\AppData\Local\Temp\ipykernel_16196\3373048541.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Sale Price'] = df['Sale Price'].str.replace('$','')


AttributeError: Can only use .str accessor with string values!

In [79]:
df.rename(columns={'Sugar Content' : 'Sugar Content g/L'})

,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Image,Link,Alcohol/Vol,Made In,Sugar Content g/L,Varietal,By,Release Date,Lat,Lon,Savings
0,Paul Mas Cabernet Sauvignon,14.95,12.95,N/A,(0),750 ml bottle,Paul Mas vineyards are in the Languedoc region...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/paul-mas-cabernet-sauv...,0.135,"Languedoc, France",5 g/L,Cabernet Sauvignon,NaN,NaN,47.218868,5.965036,2.0
1,Blu Giovello Prosecco,18.00,15.50,4.4,(9),750 ml bottle,The prosecco in the distinctive blue bottle. I...,White Sparkling Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/blu-giovello-prosecco-...,0.110,Italy,14 g/L,Prosecco,Blu Giovello,NaN,42.638426,12.674297,2.5
2,Bask Cabernet Sauvignon,13.45,12.45,3.0,(6),750 ml bottle,Low in sugar and high in flavour! Bask created...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/bask-cabernet-sauvigno...,0.125,Canada,3 g/L,Cabernet Sauvignon,NaN,NaN,61.066692,-107.991707,1.0
3,Jacob's Creek Moscato Rosé,16.00,14.00,4.7,(18),750 ml bottle,"Jacob's Creek Moscato is a fresh and fragrant,...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/jacob-s-creek-moscato-...,0.075,Australia,70 g/L,Rose Wines,Jacob's Creek,NaN,-24.776109,134.755,2.0
4,Sandbanks Summer Rosé VQA,14.95,12.95,N/A,(0),750 ml bottle,"Established in Prince Edward County in 2001, S...",Rose Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/sandbanks-summer-ros-c...,0.116,"Ontario, Canada",27 g/L,Rosé,Sandbanks,NaN,50.000678,-86.000977,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Negrar Valpolicella DOC,26.10,22.10,N/A,(0),1500 ml bottle,This versatile red wine hails from the world-f...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/negrar-valpolicella-do...,0.120,"Veneto, Italy",8 g/L,Valpolicella,Negrar,NaN,45.647666,11.866525,4.0
208,Ruffino Il Ducale Toscana IGP,0,0,5.0,(1),750 ml bottle,An innovative blend of the traditional sangiov...,Red Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/ruffino-il-ducale-tosc...,0.145,"Tuscany, Italy",7 g/L,Sangiovese,Il Ducale Igt*,NaN,43.458654,11.13892,0.0
209,Trinity Oaks Chardonnay,17.95,14.95,4.3,(3),750 ml bottle,"Partnering with Trees for the Future, a nonpro...",White Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/trinity-oaks-chardonna...,0.135,"California, United States",7 g/L,Chardonnay,Trinity Oaks,NaN,36.701463,-118.755997,3.0
210,Gran Passione Prosecco DOC Spumante,18.00,15.00,5.0,(3),750 ml bottle,Botter is a key player in the Italian wine exp...,White Sparkling Wine,https://aem.lcbo.com/content/dam/lcbo/products...,https://www.lcbo.com/en/gran-passione-prosecco...,0.110,"Veneto, Italy",14 g/L,Sparkling White,Gran Passione,NaN,45.647666,11.866525,3.0


In [ ]:
nan_df = df.isna()
df = df.fillna('N/A')
df.head()

In [ ]:
column_lists = {column: df[column].tolist() if not np.issubdtype(df[column].dtype, np.number) else df[column].fillna('N/A').tolist() for column in df.columns}

name_list = column_lists['Name']
regular_price_list = column_lists['Regular Price']
sale_price_list = column_lists['Sale Price']
rating_list = column_lists['Rating']
reviews_list = column_lists['Reviews']
size_list = column_lists['Size']
description_list = column_lists['Description']
category_list = column_lists['Category']
alcohol_list = column_lists['Alcohol/Vol']
madein_list = column_lists['Made In']
by_list = column_lists['By']
varietal_list = column_lists['Varietal']
lat_list = column_lists['Lat']
lon_list = column_lists['Lon']
savings_list = column_lists['Savings']
images_list = column_lists['Image']

lcbo_1 = {
    'name': name_list,
    'regular_price': regular_price_list,
    'sale_price': sale_price_list,
    'rating': rating_list,
    'reviews': reviews_list,
    'size': size_list,
    'description': description_list,
    'category': category_list,
    'alcohol_vol': alcohol_list,
    'made_in': madein_list,
    'by': by_list,
    'varietal': varietal_list,
    'lat': lat_list,
    'lon': lon_list,
    'savings': savings_list,
    'image': images_list
}

with open('lcbo_1.json', 'w') as file:
    json.dump(lcbo_1, file)

In [ ]:
column_lists = {column: df[column].tolist() if not np.issubdtype(df[column].dtype, np.number) else df[column].fillna('N/A').tolist() for column in df.columns}

name_list = column_lists['Name']
regular_price_list = column_lists['Regular Price']
sale_price_list = column_lists['Sale Price']
rating_list = column_lists['Rating']
reviews_list = column_lists['Reviews']
size_list = column_lists['Size']
description_list = column_lists['Description']
category_list = column_lists['Category']
alcohol_list = column_lists['Alcohol/Vol']
madein_list = column_lists['Made In']
by_list = column_lists['By']
varietal_list = column_lists['Varietal']
lat_list = column_lists['Lat']
lon_list = column_lists['Lon']
savings_list = column_lists['Savings']
images_list = column_lists['Image']

lcbo_2 = {
    'name': name_list,
    'regular_price': regular_price_list,
    'sale_price': sale_price_list,
    'rating': rating_list,
    'reviews': reviews_list,
    'size': size_list,
    'description': description_list,
    'category': category_list,
    'alcohol_vol': alcohol_list,
    'made_in': madein_list,
    'by': by_list,
    'varietal': varietal_list,
    'lat': lat_list,
    'lon': lon_list,
    'savings': savings_list,
    'image': images_list
}

with open('lcbo_2.json', 'w') as file:
    file.write(pd.DataFrame(lcbo_2).to_json(orient='records'))

In [ ]:
with open('lcbo_1.json', 'r') as file1:
    lcbo1 = json.load(file1)

# Read the contents of the second JSON file
with open('lcbo_2.json', 'r') as file2:
    lcbo2 = json.load(file2)

# Merge the two JSON data

merged_data = lcbo2
merged_data.extend(lcbo1)

# Write the merged data to a new JSON file
with open('lcbo_merged.json', 'w') as outfile:
    json.dump(merged_data, outfile)

In [80]:
df.to_csv('lcbo_wines.csv', index = False)